In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from math import sqrt

from tensorflow import keras
from tensorflow.keras.models import Sequential
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error

In [2]:
df= pd.read_csv('../power/train.csv')

In [3]:
df['강수량(mm)'] = df['강수량(mm)'].fillna(0.0)

In [4]:
df.loc[df["num_date_time"] == "15_20220806 17", "풍속(m/s)"] = (2.7 + 2.4) / 2

In [5]:
df.loc[df["num_date_time"] == "26_20220627 16", "풍속(m/s)"] = (1.0 + 0.75) / 2

In [6]:
df.loc[df["num_date_time"] == "97_20220806 15", "풍속(m/s)"] = (5.0 + 3.0) / 2
df.loc[df["num_date_time"] == "97_20220806 16", "풍속(m/s)"] = (5.0 + 3.0) / 2

In [7]:
df.loc[df["num_date_time"] == "9_20220614 11", "풍속(m/s)"] = (3.8 + 4.6) / 2

In [8]:
df.loc[df["num_date_time"] == "16_20220803 15", "풍속(m/s)"] = (3.9 + 3.8) / 2

In [9]:
df.loc[df["num_date_time"] == "26_20220627 17", "풍속(m/s)"] = (1.0 + 0.5) / 2

In [10]:
df.loc[df["num_date_time"] == "26_20220709 09", "풍속(m/s)"] = (0.0 + 1.0) / 2

In [11]:
df.loc[df["num_date_time"] == "42_20220703 10", "풍속(m/s)"] = (0.9 + 1.7) / 2

In [12]:
df.loc[df["num_date_time"] == "50_20220703 10", "풍속(m/s)"] = (0.9 + 1.7) / 2

In [13]:
df.loc[df["num_date_time"] == "52_20220806 15", "풍속(m/s)"] = (5.0 + 3.0) / 2
df.loc[df["num_date_time"] == "52_20220806 16", "풍속(m/s)"] = (5.0 + 3.0) / 2

In [14]:
df.loc[df["num_date_time"] == "87_20220714 05", "풍속(m/s)"] = (1.4 + 1.3) / 2
df.loc[df["num_date_time"] == "87_20220714 06", "풍속(m/s)"] = (1.4 + 1.3) / 2
df.loc[df["num_date_time"] == "87_20220714 07", "풍속(m/s)"] = (1.4 + 1.3) / 2
df.loc[df["num_date_time"] == "87_20220714 08", "풍속(m/s)"] = (1.4 + 1.3) / 2
df.loc[df["num_date_time"] == "87_20220714 09", "풍속(m/s)"] = (1.4 + 1.3) / 2

In [15]:
df.loc[df["num_date_time"] == "90_20220806 17", "풍속(m/s)"] = (1.3 + 2.7) / 2

In [16]:
df.loc[df["num_date_time"] == "90_20220806 15", "풍속(m/s)"] = (5.0 + 3.0) / 2
df.loc[df["num_date_time"] == "90_20220806 16", "풍속(m/s)"] = (5.0 + 3.0) / 2

In [17]:
df.loc[df["num_date_time"] == "100_20220608 15", "풍속(m/s)"] = (1.9 + 1.6) / 2

In [18]:
df.loc[df["num_date_time"] == "100_20220723 03", "풍속(m/s)"] = (0.0 + 0.0) / 2

In [19]:
df.loc[df["num_date_time"] == "9_20220614 11", "습도(%)"] = (94.0 + 91.0) / 2

In [20]:
df.loc[df["num_date_time"] == "9_20220614 11", "습도(%)"] = (54.0 + 92.0) / 2

In [21]:
df.loc[df["num_date_time"] == "87_20220714 05", "습도(%)"] = (100.0 + 90.0) / 2
df.loc[df["num_date_time"] == "87_20220714 06", "습도(%)"] = (100.0 + 90.0) / 2
df.loc[df["num_date_time"] == "87_20220714 07", "습도(%)"] = (100.0 + 90.0) / 2
df.loc[df["num_date_time"] == "87_20220714 08", "습도(%)"] = (100.0 + 90.0) / 2
df.loc[df["num_date_time"] == "87_20220714 09", "습도(%)"] = (100.0 + 90.0) / 2

In [22]:
df.loc[df["num_date_time"] == "90_20220806 17", "습도(%)"] = (54.0 + 56.0) / 2

In [23]:
df.loc[df["num_date_time"] == "100_20220608 15", "습도(%)"] = (52.0 + 53.0) / 2

In [24]:
df.loc[df["num_date_time"] == "15_20220806 17", "습도(%)"] = (54.0 + 56.0) / 2

In [25]:
df['일조(hr)'].fillna(round(df['일조(hr)'].mean(),2), inplace=True)

In [26]:
df['일사(MJ/m2)'].fillna(round(df['일사(MJ/m2)'].mean(),2), inplace=True)

In [27]:
df = df.drop(['num_date_time'], axis=1)

In [28]:
df['연도'] = df['일시'].apply(lambda x : int(x[:4]))
df['월'] = df['일시'].apply(lambda x : int(x[4:6]))
df['일'] = df['일시'].apply(lambda x : int(x[6:8]))
df['시간'] = df['일시'].apply(lambda x : int(x[9:11]))

In [29]:
df = df.drop(['일시','연도'], axis=1)

In [30]:
df = df[["월","일", "시간", "건물번호","기온(C)", "강수량(mm)", "풍속(m/s)", "습도(%)", "일조(hr)", "일사(MJ/m2)","전력소비량(kWh)"]]
df.head()

,월,일,시간,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,6,1,0,1,18.6,0.0,0.9,42.0,0.33,1.06,1085.28
1,6,1,1,1,18.0,0.0,1.1,45.0,0.33,1.06,1047.36
2,6,1,2,1,17.7,0.0,1.5,45.0,0.33,1.06,974.88
3,6,1,3,1,16.7,0.0,1.4,48.0,0.33,1.06,953.76
4,6,1,4,1,18.4,0.0,2.8,43.0,0.33,1.06,986.40


In [31]:
# 정규화하기
# MinMaxScaler 클래스 생성
scaler = MinMaxScaler()

# MinMaxScaler 클래스의 fit_transform() 메서드 사용 데이터 정규화
normalized_data = scaler.fit_transform(df)

# 정규화된 데이터, 데이터 프레임으로 다시 변환
ndf = pd.DataFrame(normalized_data, columns=df.columns)

In [32]:
X = ndf.iloc[:, :-1]
y = ndf.iloc[:,[-1]]

In [33]:
X.head()

,월,일,시간,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2)
0,0.0,0.0,0.000000,0.0,0.314815,0.0,0.067669,0.333333,0.33,0.270408
1,0.0,0.0,0.043478,0.0,0.292593,0.0,0.082707,0.367816,0.33,0.270408
2,0.0,0.0,0.086957,0.0,0.281481,0.0,0.112782,0.367816,0.33,0.270408
3,0.0,0.0,0.130435,0.0,0.244444,0.0,0.105263,0.402299,0.33,0.270408
4,0.0,0.0,0.173913,0.0,0.307407,0.0,0.210526,0.344828,0.33,0.270408


In [34]:
X = X.drop(['일조(hr)','일사(MJ/m2)'], axis=1)

In [35]:
y.head()

,전력소비량(kWh)
0,0.042579
1,0.041092
2,0.038248
3,0.037419
4,0.038700


In [36]:
'''
param_grid = {
    "max_depth": [10,30,50],
    "n_estimators": [50,100],
     "min_samples_split":[5,2],
     "min_samples_leaf":[1,2]
}
'''

'\nparam_grid = {\n    "max_depth": [10,30,50],\n    "n_estimators": [50,100],\n     "min_samples_split":[5,2],\n     "min_samples_leaf":[1,2]\n}\n'

In [37]:
'''clf = GridSearchCV(RandomForestRegressor(), param_grid, cv=5)'''

'clf = GridSearchCV(RandomForestRegressor(), param_grid, cv=5)'

In [38]:
'''clf.fit(X, y)'''

'clf.fit(X, y)'

In [39]:
'''print(clf.best_params_)'''

'print(clf.best_params_)'

In [40]:
'''param_grid = {
    "max_depth": [5,10,15],
    "n_estimators": [100,150,300],
     "min_samples_split":[1,2,3],
     "min_samples_leaf":[2,3]
}'''

'param_grid = {\n    "max_depth": [5,10,15],\n    "n_estimators": [100,150,300],\n     "min_samples_split":[1,2,3],\n     "min_samples_leaf":[2,3]\n}'

In [41]:
'''clf = GridSearchCV(RandomForestRegressor(), param_grid, cv=5)'''

'clf = GridSearchCV(RandomForestRegressor(), param_grid, cv=5)'

In [42]:
'''clf.fit(X, y)'''

'clf.fit(X, y)'

In [43]:
'''print(clf.best_params_)'''

'print(clf.best_params_)'

In [44]:
'''param_grid = {
    "max_depth": [3,5,7],
    "n_estimators": [120,150,170],
     "min_samples_split":[3,4,5],
     "min_samples_leaf":[2,10]
}'''

'param_grid = {\n    "max_depth": [3,5,7],\n    "n_estimators": [120,150,170],\n     "min_samples_split":[3,4,5],\n     "min_samples_leaf":[2,10]\n}'

In [45]:
'''clf = GridSearchCV(RandomForestRegressor(), param_grid, cv=5)'''

'clf = GridSearchCV(RandomForestRegressor(), param_grid, cv=5)'

In [46]:
'''clf.fit(X, y)'''

'clf.fit(X, y)'

In [47]:
'''print(clf.best_params_)'''

'print(clf.best_params_)'

In [48]:
'''param_grid = {
    "max_depth": [4,5,6],
    "n_estimators": [140,150,160],
     "min_samples_split":[5,10,15],
     "min_samples_leaf":[8,10,15,20]
}
'''

'param_grid = {\n    "max_depth": [4,5,6],\n    "n_estimators": [140,150,160],\n     "min_samples_split":[5,10,15],\n     "min_samples_leaf":[8,10,15,20]\n}\n'

In [49]:
'''clf = GridSearchCV(RandomForestRegressor(), param_grid, cv=5)'''

'clf = GridSearchCV(RandomForestRegressor(), param_grid, cv=5)'

In [50]:
'''clf.fit(X, y)'''

'clf.fit(X, y)'

In [51]:
'''print(clf.best_params_)'''

'print(clf.best_params_)'

In [52]:
'''param_grid = {
    "max_depth": [6,10,15],
    "n_estimators": [145,150,155],
     "min_samples_split":[9,10,11],
     "min_samples_leaf":[20,30,40,50]
}'''

'param_grid = {\n    "max_depth": [6,10,15],\n    "n_estimators": [145,150,155],\n     "min_samples_split":[9,10,11],\n     "min_samples_leaf":[20,30,40,50]\n}'

In [53]:
'''clf = GridSearchCV(RandomForestRegressor(), param_grid, cv=5)'''

'clf = GridSearchCV(RandomForestRegressor(), param_grid, cv=5)'

In [54]:
'''clf.fit(X, y)'''

'clf.fit(X, y)'

In [55]:
'''print(clf.best_params_)'''

'print(clf.best_params_)'

In [56]:
n_features = int(sqrt(df.shape[1]))
model = RandomForestRegressor(n_estimators=150, max_depth=6, min_samples_split=20, min_samples_leaf=10, max_features=n_features)

In [57]:
model = RandomForestRegressor()

In [ ]:
model.fit(X, y)

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_15880\2489992722.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y)


In [ ]:
test = pd.read_csv('../power/test.csv')

In [ ]:
test.head()

In [ ]:
test['연도'] = test['일시'].apply(lambda x : int(x[:4]))
test['월'] = test['일시'].apply(lambda x : int(x[4:6]))
test['일'] = test['일시'].apply(lambda x : int(x[6:8]))
test['시간'] = test['일시'].apply(lambda x : int(x[9:11]))

In [ ]:
test = test.drop(['일시','num_date_time','연도'], axis=1)

In [ ]:
test.head()

In [ ]:
test = test[["월","일", "시간", "건물번호","기온(C)", "강수량(mm)", "풍속(m/s)", "습도(%)"]]

In [ ]:
# 정규화하기
# MinMaxScaler 클래스 생성
scaler = MinMaxScaler()

# MinMaxScaler 클래스의 fit_transform() 메서드 사용 데이터 정규화
normalized_data = scaler.fit_transform(test)

# 정규화된 데이터, 데이터 프레임으로 다시 변환
tndf = pd.DataFrame(normalized_data, columns=test.columns)

In [ ]:
#예측하기
test_pred = model.predict(tndf)

In [ ]:
tndf['pred']=test_pred

In [ ]:
# 역정규화
min_value = 0.0
max_value = 25488.4
tndf['real_pred'] = tndf['pred'] * (max_value - min_value) + min_value 

In [ ]:
#제출
submission = pd.read_csv('../power/sample_submission.csv')
submission['answer'] = tndf['real_pred']
submission.head()

In [ ]:
submission.to_csv('../power/power9.csv', index=  False)